$\newcommand{\To}{\Rightarrow}$
$\newcommand{\false}{\mathrm{false}}$

In [1]:
import os, sys
sys.path.append(os.path.split(os.getcwd())[0])

In [2]:
from kernel.type import boolT
from kernel.term import Var
from kernel.thm import Thm
from kernel.report import ProofReport
from logic import basic
from logic import matcher
from logic.logic import conj, disj
from logic.logic_macro import apply_theorem
from logic.proofterm import ProofTerm, ProofTermDeriv, ProofTermMacro
from syntax import printer

thy = basic.loadTheory('logic_base')

## Macros

In this section, we introduce the central concept of macros. Macros are deduction rules that serve as abbreviations for more elementary proof steps. In both linear proof and proof terms, one can use an invocation of a macro to substitute for multiple steps of proof. The macro invocations can be expanded on demand during proof checking. Hence, they shorten the length of proofs that need to be stored, and make the whole system scalable to large proofs.

The standard way to create a macro is to inherit from class `ProofTermMacro` (so called because it constructs a macro from functions working with proof terms). The new macro class needs to implement the `get_proof_term` method, which takes as arguments the current theory, arguments to the macro, and a list of input proof terms. If the inputs are valid, the function should return the resulting proof term.

Again, we use applying a theorem as an example, this time implementing it as a macro. We choose the name `apply_theorem_test` to avoid name conflicts with the existing `apply_theorem` macro.

In [3]:
class apply_theorem_test(ProofTermMacro):
    def get_proof_term(self, thy, th_name, prevs):
        pt = ProofTerm.theorem(thy, th_name)
        As, _ = pt.prop.strip_implies()  # list of assumptions of pt
        instsp = dict(), dict()      # initial (empty) instantiation
        for A, prev in zip(As, prevs):   # match each assumption with corresponding prev
            matcher.first_order_match_incr(A, prev.prop, instsp)
        tyinst, inst = instsp
        pt2 = ProofTerm.subst_type(tyinst, pt) if tyinst else pt   # perform substitution on pt
        pt3 = ProofTerm.substitution(inst, pt2) if inst else pt2
        for prev in prevs:                   # perform implies_elim on ptm
            pt3 = ProofTerm.implies_elim(pt3, prev)
        return pt3

The implementation of `get_proof_term` in `apply_theorem_test` is exactly the same as `apply_theorem` function in the previous section. We can test this macro independently:

In [4]:
A = Var("A", boolT)
B = Var("B", boolT)
ptA = ProofTerm.assume(A)
ptB = ProofTerm.assume(B)
ptAB = apply_theorem_test().get_proof_term(thy, 'conjI', [ptA, ptB])
prf = ptAB.export()
thy.check_proof(prf)
print(printer.print_proof(thy, prf, unicode=True))

0: ⊢ A ⟶ B ⟶ A ∧ B by theorem conjI
1: ⊢ A ⟶ B ⟶ A ∧ B by substitution {A: A, B: B} from 0
2: A ⊢ A by assume A
3: A ⊢ B ⟶ A ∧ B by implies_elim from 1, 2
4: B ⊢ B by assume B
5: A, B ⊢ A ∧ B by implies_elim from 3, 4


To use this macro in proofs, we need to first register it to the theory object. For this, we use the `add_proof_macro` method. It takes two arguments: the name of the macro, and the macro object.

In [5]:
thy.add_proof_macro('apply_theorem_test', apply_theorem_test())

Even though the name of the macro agrees with the class name in this case, this is not required. Nor is it necessary that the macro object be built from a constructor taking no arguments. It can be built in any other way, as long as the `get_proof_term` method is present. This also marks the first time that we modified the theory object directly. As it is now clear, it is the job of the theory object to keep track of the set of macros that can be used in proofs, in addition to the list of types, constants, theorems, and so on.

In [6]:
ptA = ProofTerm.assume(A)
ptB = ProofTerm.assume(B)
ptAB = ProofTermDeriv('apply_theorem_test', thy, 'conjI', prevs=[ptA, ptB])
print(printer.print_thm(thy, ptAB, unicode=True))

A, B ⊢ A ∧ B


As we can see, the sequent obtained by `ptAB` is computed immediately, using the `get_proof_term` function. However, the full proofs is not stored:

In [7]:
prf = ptAB.export()
print(printer.print_proof(thy, prf, unicode=True))

0: assume A
1: assume B
2: apply_theorem_test conjI from 0, 1


In the exported proof, the proof rule `apply_theorem_test` appears directly. Let us now check the proof while keeping the report:

In [8]:
rpt = ProofReport()
thy.check_proof(prf, rpt)
print(printer.print_proof(thy, prf, unicode=True), "\n")
print(rpt)

0: A ⊢ A by assume A
1: B ⊢ B by assume B
2: A, B ⊢ A ∧ B by apply_theorem_test conjI from 0, 1 

Steps: 6
  Theorems:  1
  Primitive: 5
  Macro:     0
Theorems applied: conjI
Macros evaluated: 
Macros expanded: apply_theorem_test
Gaps: []


This time the report is quite informative. While the original proof is only 3 steps, the report says 6 steps were taken, of which there is one application of a theorem (conjI) and 5 primitive steps. In addition, the macro `apply_theorem_test` is expanded. This means during proof checking, the full proof behind `apply_theorem_test` is expanded, first using its `get_proof_term` function to obtain a proof term, then converting that proof term into a linear proof.

## Trusted macros

So far during proof checking, we always expanded invocations of macros into full proofs. In this way, we do not have to trust that the implementations of macros are correct: if there is any bug in the implementation, an exception will be raised when generating the full proof, or the generated proof will be incorrect and cannot pass proof-checking. Hence, always expanding macros give a very high level of confidence in the proofs. However, this level of confidence has a cost: often the sequent obtained by the macro can be computed much more efficiently than generating the full proof. Hence, it is often helpful to directly evaluate the sequent obtained by the macro, without generating full proofs. This lowers the level of confidence we have in the proof (if there is any bug in the evaluation function, it can go undetected and result in an incorrect proof), but the increase in efficiency can be substantial.

For a macro to be evaluated directly, it has to provide two additional pieces of information: its trust level (given by field `level`) and its evaluation function (given by method `eval`). As an example, we fill in these two functions for the macro `apply_theorem_test`:

In [9]:
class apply_theorem_test(ProofTermMacro):
    def __init__(self):
        self.level = 1
        
    def eval(self, thy, th_name, prevs):
        print("Calling eval...")
        th = thy.get_theorem(th_name)
        As, C = th.prop.strip_implies()
        instsp = dict(), dict()
        for A, prev in zip(As, prevs):
            matcher.first_order_match_incr(A, prev.prop, instsp)
        tyinst, inst = instsp
        prev_hyps = sum([prev.hyps for prev in prevs], ())
        C2 = C.subst_type(tyinst).subst(inst)
        return Thm(th.hyps + prev_hyps, C2)

    def get_proof_term(self, thy, th_name, prevs):
        print("Calling get_proof_term...")
        pt = ProofTerm.theorem(thy, th_name)
        As, _ = pt.prop.strip_implies()  # list of assumptions of pt
        instsp = dict(), dict()      # initial (empty) instantiation
        for A, prev in zip(As, prevs):   # match each assumption with corresponding prev
            matcher.first_order_match_incr(A, prev.prop, instsp)
        tyinst, inst = instsp
        pt2 = ProofTerm.subst_type(tyinst, pt) if tyinst else pt   # perform substitution on pt
        pt3 = ProofTerm.substitution(inst, pt2) if inst else pt2
        for prev in prevs:                   # perform implies_elim on ptm
            pt3 = ProofTerm.implies_elim(pt3, prev)
        return pt3

We added two calls to print so we can see which methods are invoked at what time. First, we add the new macro to the theory (which overwrites the existing macro).

In [10]:
thy.add_proof_macro('apply_theorem_test', apply_theorem_test())

Next, we test with a simple example:

In [11]:
ptA = ProofTerm.assume(A)
ptB = ProofTerm.assume(B)
ptAB = ProofTermDeriv('apply_theorem_test', thy, 'conjI', prevs=[ptA, ptB])
print(printer.print_thm(thy, ptAB, unicode=True))

Calling eval...
A, B ⊢ A ∧ B


Note that the evaluation function is called in `ProofTermDeriv` to generate the sequent obtained by the macro. This is always the case when `eval` is provided, since we do not need to trust anything at this point, because the proof still needs to be checked later. In proof checking, we can specify an additional argument `check_level`. During proof checking, all macros with level less or equal to the check level do not need to be expanded (if a macro does not have level specified, it is always expanded). For example:

In [12]:
rpt = ProofReport()
thy.check_proof(prf, rpt, check_level=1)
print(printer.print_proof(thy, prf, unicode=True), "\n")
print(rpt)

Calling eval...
0: A ⊢ A by assume A
1: B ⊢ B by assume B
2: A, B ⊢ A ∧ B by apply_theorem_test conjI from 0, 1 

Steps: 3
  Theorems:  0
  Primitive: 2
  Macro:     1
Theorems applied: 
Macros evaluated: apply_theorem_test
Macros expanded: 
Gaps: []


Note the evaluation function is called, and the report indicates there are only three steps in the proof. The report also shows `apply_theorem_test` is among the set of macros that are evaluated. Also note that proof checking did not detect the use of theorem `conjI`. This is because the invocation of the theorem lies within the macro expansion.

We now consider a more complete example, showing commutativity of conjunction:

In [13]:
pt0 = ProofTerm.assume(conj(A, B))
pt1 = ProofTermDeriv('apply_theorem_test', thy, 'conjD1', [pt0])
pt2 = ProofTermDeriv('apply_theorem_test', thy, 'conjD2', [pt0])
pt3 = ProofTermDeriv('apply_theorem_test', thy, 'conjI', [pt2, pt1])
pt4 = ProofTerm.implies_intr(conj(A, B), pt3)
print(printer.print_thm(thy, pt4.th, unicode=True))

Calling eval...
Calling eval...
Calling eval...
⊢ A ∧ B ⟶ B ∧ A


Note the evaluation function is called at every call of `ProofTermDeriv`. Now, we check the proof, first with macro expansion:

In [14]:
prf = pt4.export()
rpt = ProofReport()
thy.check_proof(prf, rpt)
print(rpt)

Calling get_proof_term...
Calling get_proof_term...
Calling get_proof_term...
Steps: 12
  Theorems:  3
  Primitive: 9
  Macro:     0
Theorems applied: conjD1, conjI, conjD2
Macros evaluated: 
Macros expanded: apply_theorem_test
Gaps: []


We see all three calls to `apply_theorem_test` are expanded. The resulting proof has 12 steps in total. Next, we check the proof with macro expansion:

In [15]:
rpt2 = ProofReport()
thy.check_proof(prf, rpt2, check_level=1)
print(rpt2)

Calling eval...
Calling eval...
Calling eval...
Steps: 5
  Theorems:  0
  Primitive: 2
  Macro:     3
Theorems applied: 
Macros evaluated: apply_theorem_test
Macros expanded: 
Gaps: []


We see that all three calls to `apply_theorem_test` are simply evaluated, and the report is quite different, showing only 5 steps of proof. Printing the proof itself also shows only 5 steps:

In [16]:
print(printer.print_proof(thy, prf, unicode=True))

0: A ∧ B ⊢ A ∧ B by assume A ∧ B
1: A ∧ B ⊢ B by apply_theorem_test conjD2 from 0
2: A ∧ B ⊢ A by apply_theorem_test conjD1 from 0
3: A ∧ B ⊢ B ∧ A by apply_theorem_test conjI from 1, 2
4: ⊢ A ∧ B ⟶ B ∧ A by implies_intr A ∧ B from 3


## Applying theorems in practice

The actual `apply_theorem` macro has more features: it allows two versions - with and without specifying an instantiation. The version taking an instantiation is used when first-order matching is unable to produce assignments for all variables. This may be because there are variables in the conclusion but not in the assumptions of the theorem, or because applying the theorem requires higher-order matching. The macro not taking instantiations is called `apply_theorem` and the macro taking instantiations is called `apply_theorem_for`.

Invocations to both macros are wrapped in a function `apply_theorem`. We now demonstrate the use of this function.

In [17]:
pt0 = ProofTerm.assume(conj(A, B))
pt1 = apply_theorem(thy, 'conjD1', pt0)
pt2 = apply_theorem(thy, 'conjD2', pt0)
pt3 = apply_theorem(thy, 'conjI', pt2, pt1)
pt4 = ProofTerm.implies_intr(conj(A, B), pt3)
prf = pt4.export()
thy.check_proof(prf)
print(printer.print_proof(thy, prf, unicode=True))

0: A ∧ B ⊢ A ∧ B by assume A ∧ B
1: A ∧ B ⊢ B by apply_theorem conjD2 from 0
2: A ∧ B ⊢ A by apply_theorem conjD1 from 0
3: A ∧ B ⊢ B ∧ A by apply_theorem conjI from 1, 2
4: ⊢ A ∧ B ⟶ B ∧ A by implies_intr A ∧ B from 3


Next, we consider the proof of commutativity of disjunction, which requires specifying the conclusion (or a partial instantiation) in some calls to `apply_theorem`.

In [18]:
pt0 = ProofTerm.assume(disj(A, B))
pt1 = ProofTerm.assume(A)
pt2 = apply_theorem(thy, 'disjI2', pt1, concl=disj(B, A))
pt3 = ProofTerm.implies_intr(A, pt2)
pt4 = ProofTerm.assume(B)
pt5 = apply_theorem(thy, 'disjI1', pt4, concl=disj(B, A))
pt6 = ProofTerm.implies_intr(B, pt5)
pt7 = apply_theorem(thy, 'disjE', pt0, pt3, pt6)
pt8 = ProofTerm.implies_intr(disj(A, B), pt7)
prf = pt8.export()
thy.check_proof(prf)
print(printer.print_proof(thy, prf, unicode=True))

0: A ∨ B ⊢ A ∨ B by assume A ∨ B
1: A ⊢ A by assume A
2: A ⊢ B ∨ A by apply_theorem_for disjI2, {}, {A: B, B: A} from 1
3: ⊢ A ⟶ B ∨ A by implies_intr A from 2
4: B ⊢ B by assume B
5: B ⊢ B ∨ A by apply_theorem_for disjI1, {}, {A: B, B: A} from 4
6: ⊢ B ⟶ B ∨ A by implies_intr B from 5
7: A ∨ B ⊢ B ∨ A by apply_theorem disjE from 0, 3, 6
8: ⊢ A ∨ B ⟶ B ∨ A by implies_intr A ∨ B from 7


Checking the proof by expanding the macros yields the following report:

In [19]:
rpt = ProofReport()
thy.check_proof(prf, rpt)
print(rpt)

Steps: 20
  Theorems:  3
  Primitive: 17
  Macro:     0
Theorems applied: disjE, disjI2, disjI1
Macros evaluated: 
Macros expanded: apply_theorem, apply_theorem_for
Gaps: []
